In [20]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

In [21]:
import sys
print(sys.path) 
sys.path.append("/cellar/users/mpagadal/Programs/anaconda3/lib/python3.7/site-packages")
sys.path.insert(1, '/cellar/users/mpagadal/Data/gcta/scripts')
import json

['/data/cellardata/users/mpagadal/gcta/notebooks', '/cellar/users/mpagadal/Data/gcta/scripts', '/nrnb/opt/anaconda3.7/lib/python37.zip', '/nrnb/opt/anaconda3.7/lib/python3.7', '/nrnb/opt/anaconda3.7/lib/python3.7/lib-dynload', '', '/cellar/users/mpagadal/.local/lib/python3.7/site-packages', '/nrnb/opt/anaconda3.7/lib/python3.7/site-packages', '/nrnb/opt/anaconda3.7/lib/python3.7/site-packages/IPython/extensions', '/cellar/users/mpagadal/.ipython', '/cellar/users/mpagadal/Programs/anaconda3/lib/python3.7/site-packages']


In [22]:
from compile_gcta_df import *

## Get nonhla and nonkir data

In [23]:
def fdr(df,pseudo):
    #make heritability estimate numeric
    df["V(G)/Vp"]=pd.to_numeric(df["V(G)/Vp"])
    #add pseudocount
    if pseudo == True:
        df["Pval"]=df["Pval"]+0.0000001
    
    #calculate benjamin-hochberg fdr
    genes=df["gene"].tolist()
    probabilities = df["Pval"].to_numpy()
    report = multi.multipletests(probabilities,
                alpha=0.05,
                method="fdr_bh",
                is_sorted=False,
            )
    discoveries = report[1]
    df["fdr"]=discoveries
    df["fdr"]=pd.to_numeric(df["fdr"])
    df["-log10(fdr)"]=-np.log10(df["fdr"])
    
    return(df)

In [24]:
results=[""]

In [25]:
nonhla_brian=compile_gcta_results(results,output="overall",direct="/cellar/users/mpagadal/Data/gcta/results/tcga/brian-tpm/no-hla-kir")
nonhla_firebrowse=compile_gcta_results(results,output="overall",direct="/cellar/users/mpagadal/Data/gcta/results/tcga/FireBrowse/no-hla-kir")
nonhla_pancan=compile_gcta_results(results,output="overall",direct="/cellar/users/mpagadal/Data/gcta/results/tcga/PanCanAtlas/no-hla-kir")

compiling gene level heritability dataframe.....
compiling gene level heritability dataframe.....
compiling gene level heritability dataframe.....


In [26]:
hla_brian=compile_gcta_results(results,output="overall",direct="/cellar/users/mpagadal/Data/gcta/results/tcga/brian-tpm/hla-kir")
hla_firebrowse=compile_gcta_results(results,output="overall",direct="/cellar/users/mpagadal/Data/gcta/results/tcga/FireBrowse/hla-kir")
hla_pancan=compile_gcta_results(results,output="overall",direct="/cellar/users/mpagadal/Data/gcta/results/tcga/PanCanAtlas/hla-kir")

compiling gene level heritability dataframe.....
compiling gene level heritability dataframe.....
compiling gene level heritability dataframe.....


In [27]:
print(nonhla_brian.shape)
print(nonhla_firebrowse.shape)
print(nonhla_pancan.shape)

(746, 16)
(738, 16)
(741, 16)


In [28]:
pseudocount=True
nonhla_brian=fdr(nonhla_brian,pseudocount)
nonhla_firebrowse=fdr(nonhla_firebrowse,pseudocount)
nonhla_pancan=fdr(nonhla_pancan,pseudocount)

In [29]:
pseudocount=True
hla_brian=fdr(hla_brian,pseudocount)
hla_firebrowse=fdr(hla_firebrowse,pseudocount)
hla_pancan=fdr(hla_pancan,pseudocount)

In [30]:
#standardize values
mp_ids={"CMC2":"C16orf61",
    "ADGRE5":"CD97",
    "SPRYD7":"C13orf1",
    "JCHAIN":"IGJ",
    "CTSL":"CTSL1",
    "CTSV":"CTSL2",
     "TEX30":"C13orf27",
     "CENPU":"MLF1IP",
     "CMSS1":"C3orf26",
     "FAM216A":"C12orf24",
     "HACD2":"PTPLB",
    "MARCHF1":"MARCH1"}
mp_ids= {v: k for k, v in mp_ids.items()}

In [31]:
nonhla_brian["gene"].replace(mp_ids, inplace=True)
nonhla_firebrowse["gene"].replace(mp_ids, inplace=True)
nonhla_pancan["gene"].replace(mp_ids, inplace=True)

hla_brian["gene"].replace(mp_ids, inplace=True)
hla_firebrowse["gene"].replace(mp_ids, inplace=True)
hla_pancan["gene"].replace(mp_ids, inplace=True)

In [32]:
nonhla_brian=nonhla_brian.rename(columns={"V(G)/Vp":"brian V(G)/V(p)","-log10(fdr)":"brian -log10(fdr)"})
nonhla_firebrowse=nonhla_firebrowse.rename(columns={"V(G)/Vp":"firebrowse V(G)/V(p)","-log10(fdr)":"firebrowse -log10(fdr)"})
nonhla_pancan=nonhla_pancan.rename(columns={"V(G)/Vp":"pancan V(G)/V(p)","-log10(fdr)":"pancan -log10(fdr)"})


In [33]:
hla_brian=hla_brian.rename(columns={"V(G)/Vp":"brian V(G)/V(p)","-log10(fdr)":"brian -log10(fdr)"})
hla_firebrowse=hla_firebrowse.rename(columns={"V(G)/Vp":"firebrowse V(G)/V(p)","-log10(fdr)":"firebrowse -log10(fdr)"})
hla_pancan=hla_pancan.rename(columns={"V(G)/Vp":"pancan V(G)/V(p)","-log10(fdr)":"pancan -log10(fdr)"})


In [34]:
total_nonhla=pd.merge(nonhla_brian[["brian V(G)/V(p)","gene","brian -log10(fdr)"]],nonhla_firebrowse[["firebrowse V(G)/V(p)","gene","firebrowse -log10(fdr)"]], on="gene",how="outer")
total_nonhla=pd.merge(total_nonhla,nonhla_pancan[["pancan V(G)/V(p)","gene","pancan -log10(fdr)"]],on="gene",how="outer")


In [35]:
total_hla=pd.merge(hla_brian[["brian V(G)/V(p)","gene","brian -log10(fdr)"]],hla_firebrowse[["firebrowse V(G)/V(p)","gene","firebrowse -log10(fdr)"]], on="gene",how="outer")
total_hla=pd.merge(total_hla,hla_pancan[["pancan V(G)/V(p)","gene","pancan -log10(fdr)"]],on="gene",how="outer")


In [36]:
total_nonhla["consensus V(g)/V(p)"]=total_nonhla[["firebrowse V(G)/V(p)","pancan V(G)/V(p)"]].apply(lambda x: len([x for x in list(x) if x>0.05]), axis=1)
total_nonhla["consensus fdr"]=total_nonhla[["firebrowse -log10(fdr)","pancan -log10(fdr)"]].apply(lambda x: len([x for x in list(x) if x>1]), axis=1)


In [37]:
total_hla["consensus V(g)/V(p)"]=total_hla[["firebrowse V(G)/V(p)","pancan V(G)/V(p)"]].apply(lambda x: len([x for x in list(x) if x>0.05]), axis=1)
total_hla["consensus fdr"]=total_hla[["firebrowse -log10(fdr)","pancan -log10(fdr)"]].apply(lambda x: len([x for x in list(x) if x>1]), axis=1)


In [38]:
total_nonhla.to_csv("nonhla_gcta_results.tsv",sep="\t",index=None)
total_hla.to_csv("hla_gcta_results.tsv",sep="\t",index=None)

In [41]:
total_nonhla.head()

,brian V(G)/V(p),gene,brian -log10(fdr),firebrowse V(G)/V(p),firebrowse -log10(fdr),pancan V(G)/V(p),pancan -log10(fdr),consensus V(g)/V(p),consensus fdr
0,0.050055,PDIA3,0.664149,0.006458,0.301030,0.028543,0.324669,0,0
1,0.029649,TNFRSF9,0.572291,0.035394,0.426960,0.032488,0.420132,0,0
2,0.070778,CTSS,1.277438,0.037974,0.339425,0.057679,0.521608,1,0
3,0.000001,Proliferation,0.301030,0.000001,0.301030,0.000001,0.301030,0,0
4,0.000001,XCL1,0.301030,0.000001,0.301030,0.000001,0.301030,0,0
